### Dependencies

In [1]:
import numpy as np
import pandas as pd
from datetime import *

import matplotlib.pyplot as plt
# Set the default figure size
plt.rcParams['figure.figsize'] = (12, 8)

In [2]:
import os
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
os.chdir(parent_directory)

In [3]:
from src.data_fetch import get_daily_price
from src.preprocess import processed_data, features
from src.paths import ProjectPaths

### Data

In [4]:
symbol = 'SBIN'

In [5]:
df = processed_data(symbol)
df = df[['DATE', 'CLOSE']]
df = df.rename(columns = {'DATE': 'datetime', 'CLOSE': 'price'})
df = df.set_index('datetime').reset_index()

### Feature Engineering

In [6]:
data = features(df)

In [9]:
# dropping null values
data.dropna(inplace = True)

### Modelling

In [12]:
# training upto this date
training_upto = (datetime.now() + timedelta(days = -287)).strftime('%Y-%m-%d')

# validation data upto this date
validation_upto = (datetime.now() + timedelta(days = -143)).strftime('%Y-%m-%d')


# Validation Cut Off (training upto this date)
validation_cutoff = training_upto

# Test Cut Off (validation upto this date)
test_cutoff = validation_upto

# Feature columns
X = data.copy()

# Training set
training_data = X[X['datetime'] < validation_cutoff].copy()

# Validation set
valid_data = X[(X['datetime'] >= validation_cutoff) & (X['datetime']< test_cutoff)].copy()

# Test set
test_data = X[X['datetime'] >= test_cutoff].copy()


train = training_data.copy()
valid = valid_data.copy()
test = test_data.copy()

In [13]:
# Training set
X_train = train[train['datetime'] < validation_cutoff+ ' 00:00:00'].drop('target', axis = 1).copy().set_index('datetime')
y_train = train[train['datetime'] < validation_cutoff+ ' 00:00:00'][['datetime', 'target']].copy().set_index('datetime')

# Validation set
X_valid = valid[(valid['datetime']>= validation_cutoff+ ' 00:00:00') & (valid['datetime']< test_cutoff+ ' 00:00:00')].drop('target', axis = 1).copy().set_index('datetime')
y_valid = valid[(valid['datetime']>= validation_cutoff+ ' 00:00:00') & (valid['datetime']< test_cutoff+ ' 00:00:00')][['datetime', 'target']].copy().set_index('datetime')

# Test set
X_test = test[test['datetime'] >= test_cutoff+ ' 00:00:00'].drop('target', axis = 1).copy().set_index('datetime')
y_test = test[test['datetime'] >= test_cutoff+ ' 00:00:00'][['datetime', 'target']].copy().set_index('datetime')

In [14]:
total_data = train.shape[0] + valid.shape[0] + test.shape[0]
total_data

1232

In [15]:
train.shape[0]*100/total_data

84.41558441558442

In [16]:
valid.shape[0]*100/total_data

8.116883116883116

In [17]:
test.shape[0]*100/total_data

7.467532467532467

In [23]:
# importing lightgbm library
import lightgbm as lgb

OSError: dlopen(/Users/amanbhatt/Desktop/Stock_Analysis/stock_env/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <D21A7969-4567-3BC7-94ED-6A9E83AE9D78> /Users/amanbhatt/Desktop/Stock_Analysis/stock_env/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [22]:


# # defining parameters 
# params = {'task': 'train',   # for training
#           'boosting': 'gbdt',   # Gradient Boosting Decision Tree
#           'objective': 'regression',   # doing regression
#           'metric': 'mae',   # root mean squared error
#           'verbose': -1,
#           'categorical_feature': ''
#          }

# # training set
# lgb_train = lgb.Dataset(X_train, y_train)

# # validation set
# lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train)

# # training model with basic parameters
# model = lgb.train(params, lgb_train, valid_sets = lgb_eval, 
#                   num_boost_round = 5000, early_stopping_rounds = 10,
#                   verbose_eval = False
#                  )